In [ ]:
# Help tools
#from IPython.core.debugger import Tracer; Tracer()()
#np.set_printoptions(threshold=np.inf)

In [53]:
import random
import numpy
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier

In [42]:
chunk_size=1000
n=25667779
s=100000

In [47]:
### Exercise 1.1(b)
skip=sorted(random.sample(xrange(n),s))
skip=set(skip)
count=0
with open('../data/landsat_train.csv') as f:
  with open('../out/landsat_train_subset.csv','w') as f1:
    with open('../out/landsat_train_remaining.csv', 'w') as f2:
      for line in f:
	if count in skip:
           f1.write(line)
	   skip.remove(count)
	else:
	   f2.write(line)
	count+=1

In [50]:
### Exercise 1.2
print("Load the data...")
data_train = np.genfromtxt("../out/landsat_train_subset.csv", delimiter=",")
Xtrain, Ytrain = data_train[:,1:], data_train[:,0]

Load the data...


In [51]:
print("Train random forest...")
rf=RandomForestClassifier(n_estimators=10,criterion='gini',max_depth=None,min_samples_split=2,max_features=None)
rf.fit(Xtrain,Ytrain)
pred=rf.predict(Xtrain)

Train random forest...


In [55]:
count=0
print("Calculating misclassification error...")
for i in range(len(pred)):
    if pred[i]==Ytrain[i]:
       count+=1
error=1.0 - count/100000.0
print("Misclassification error: %f" % error)
print("Saving model...")
filename='../out/model.sav'
pickle.dump(rf,open(filename,'wb'))

Calculating misclassification error...
Misclassification error: 0.003230
Saving model...


In [59]:
### Exercise 1.3
filename='../out/model.sav'
loaded_model=pickle.load(open(filename,'rb'))
def chunk_generator(fname, chunk_size=1000):
    data = []
    counter = 0
    with open(fname, 'r') as ifile:

        for line in ifile:
            if not line.startswith("#") and counter < chunk_size:
                line = line.split(",")
                data.append([float(p) for p in line])
                counter += 1
            if counter == chunk_size:
                counter = 0
                yield data
                data = []

        # process remaining elements
        if len(data) > 0:
            yield data

In [58]:
preds=[]
Ytest=[]
for chunk in chunk_generator("../out/landsat_train_remaining.csv",chunk_size=1000):
    preds+=list(loaded_model.predict(np.array(chunk)[:,1:]))
    Ytest+=list(np.array(chunk)[:,0])
count=0
for i in range(len(preds)):
    if preds[i]==Ytest[i]:
       count+=1
error=1.0-(count/float(len(preds)))
print(error)

0.0703545661905
